In [44]:
import datetime
import os 
from dotenv import load_dotenv
import numpy as np

path = '/home/rafaelfabrichimidt/Documentos/projetos/env_vars/'
envfile = '.env'

load_dotenv(dotenv_path = path + envfile)

True

In [5]:
import duckdb
conn = duckdb.execute("ATTACH 'dbname=postgresql_oltp user=admin host=localhost, password=12345' AS db (TYPE POSTGRES); ")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
from openai import OpenAI
client = OpenAI()

In [29]:
system_prompt_sql = 'Você é responsável por construir queries SQL Postgres, para a query SQL funcionar deve-se adicionar no FROM database.schema.table. Retorne apenas a query SQL'

descricao_base_enem_sql = '''
                                database=db;
                                schema=enem;
                                table=base_enem;
                                campo nu_inscricao  tipo int é o número da inscrição do aluno; 
                                campo nu_nota_cn tipo int Nota da prova de Ciências da Natureza; 
                                campo nu_nota_ch tipo int Nota da prova de Ciências Humanas;
                                campo nu_nota_lc tipo int Nota da prova de Linguagens e Códigos;
                                campo nu_nota_mt tipo int Nota da Nota da prova de Matemática;

'''

user_mensage_question = 'Sou o aluno com inscrição 210061103945 desejava saber meu desempenho em Matemática?'



In [30]:
mensagens = [
    {'role': 'system', 'content': system_prompt_sql},
    {'role': 'system', 'content': descricao_base_enem_sql},
    {'role': 'user', 'content': user_mensage_question}
]

In [31]:
resposta = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=mensagens,
    max_tokens=50,
    temperature=0.7,
)

mensagem_resp = resposta.choices[0].message.content
print(mensagem_resp)

sql_query = mensagem_resp.replace("```", "").split('sql')[1]

```sql
SELECT nu_nota_mt 
FROM db.enem.base_enem 
WHERE nu_inscricao = 210061103945;
```


In [32]:
df = conn.sql(sql_query)

In [76]:
system_prompt_dialogue = '''
                                Você é um professor responsável por analisar o desempenho de 
                                um aluno no Enem e recomendar estudos de acordo com sua pontuação
                                um resumo das Competência de cada área, não precisa repetir as 
                                competências apenas descrever
'''


descricao_detalhada_enem = open(file = 'descricao_detalhada.txt', mode = 'r').read()

user_mensage_with_data = user_mensage_question + 'Os dados com detalhes do desempenho do Aluno encontra-se abaixo \n ' + f' {df.fetchnumpy()} Ajude o aluno para melhorar sua nota: '

In [80]:
prompt_resposta = [
    {'role': 'system', 'content': system_prompt_dialogue},
    {'role': 'system', 'content': descricao_detalhada_enem},                                
    {'role': 'user', 'content': user_mensage_with_data}
     ]

In [91]:
resposta = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompt_resposta,
    max_tokens=200,
    temperature=0.7,
)

mensagem_resp = resposta.choices[0].message.content
print(mensagem_resp)


BadRequestError: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not supported with this model. Use 'max_completion_tokens' instead.", 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': 'unsupported_parameter'}}